Setting up the 5 CNN-Based classifiers as an ensemble and using it to make predictions about whether images contain asteroid tracklets. More information in Towards Asteroid Detection in Microlensing Surveys with Deep Learning by Cowan et al.

In [1]:
import os
import numpy as np
import pickle
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

import matplotlib.image as mpimg

In [2]:
MOA12 = r"/path_to/MOA_12.h5"
MOA14 = r"/path_to/MOA_14.h5"
MOA15 = r"/path_to/MOA_15.h5"
HYBRID_A = r"/path_to/hybrid_a.h5"
HYBRID_B = r"/path_to/hybrid_b.h5"

PREDICTION_SAVE_DIR = r"/save_path/"

In [4]:
#-----------------------------------------------------------------------------------------------------------------
#ASSEMBLE ENSEMBLE
#-----------------------------------------------------------------------------------------------------------------

model_moa12 = load_model(MOA12)
model_moa14 = load_model(MOA14)
model_moa15 = load_model(MOA15)
model_hybridA = load_model(HYBRID_A)
model_hybridB = load_model(HYBRID_B)

ensemble_models = [model_moa12, model_moa14, model_moa15, model_hybridA, model_hybridB]

In [6]:
#------------------------------------------------------------------
# MAKE PREDICTIONS USING ENSEMBLE FOR A SINGLE IMAGE.
# RETURNS PREDICTION SCORE 
#------------------------------------------------------------------

def make_predictions(img_path):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array /= 255.0
    img_array = tf.expand_dims(img_array, 0) 
    predictions = []
    
    #tf prefers this format for calling the ensemble
    for model in ensemble_models:
        predict_val = model.predict(img_array)
        predictions.append(model.predict(img_array))

    #result = np.mean(predictions) # result is the average of all predictions (fewer false positives)
    result = np.max(predictions) # result is the highest value prediction (fewer false negatives)

    return result
    
# TEST SCRIPT
test_img = r"/home/pcowan/Code/Asteroid-Detection/DeepLearning/Classify_128/WithData2/2453874_1280_3840.jpg"
score = make_predictions(test_img)
print("%s is %.2f percent an mp and %.2f percent not an mp. Score is %.2f."
        % (os.path.basename(test_img), 100 * score, 100 * (1 - score),score))    

2453874_1280_3840.jpg is 100.00 percent an mp and 0.00 percent not an mp. Score is 1.00.


In [ ]:
#-----------------------------------------------------------------------
# MAKE PREDICTIONS USING ENSEMBLE FOR A LIST OF FILES AND SAVE RESULT
#-----------------------------------------------------------------------

def predict_group(flist, save_name):
    yes_tracklet = []
    no_tracklet = []

    for fname in tqdm(flist):
        score = make_predictions(fname)
    
        if (score > 0.5):
            yes_tracklet.append(fname)
        else:
            no_tracklet.append(fname)

    print("%d images from %s classified as having a tracklet." % (len(yes_tracklet), save_name))
    print("%d images from %s classified as not having a tracklet." % (len(no_tracklet), save_name))
    
    yes_save = PREDICTION_SAVE_DIR + "yes/" + save_name + "_yes.dat"
    no_save = PREDICTION_SAVE_DIR +  "no/" + save_name + "_no.dat"

    with open(yes_save, 'wb') as yesfile:
        pickle.dump(yes_tracklet, yesfile)
        
    with open(no_save, 'wb') as nofile:
        pickle.dump(no_tracklet, nofile)